In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
# program = "graph_coloring"
# program = "maximal_matching"
program = "dijkstra_token_ring"
# graph_name = "graph_7"
graph_name = "implicit_graph_n10"

no_simulations = 100000
fault_interval = 3
sched = 0

selected_nodes = [0, 1, 9]

simulation_type = "controlled_at_node_amit_v2"
limits = 100

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__{simulation_type}__{limits}"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,0,3626.647599
1,1,3537.818957
2,2,3450.606478
3,3,3364.980759
4,4,3280.912929
...,...,...
95,95,-402.405324
96,96,-417.928674
97,97,-433.169590
98,98,-448.133211


In [5]:
dfs[0]

,Steps,exp_fit
0,0,3104.865123
1,1,3040.725708
2,2,2977.487901
3,3,2915.139028
4,4,2853.666594
...,...,...
95,95,-269.020115
96,96,-285.732815
97,97,-302.210585
98,98,-318.456727


In [6]:
dfs[1]

,Steps,exp_fit
0,0,2744.287749
1,1,2698.426305
2,2,2652.960946
3,3,2607.888253
4,4,2563.204833
...,...,...
95,95,-236.524794
96,96,-256.642216
97,97,-276.585892
98,98,-296.357323


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__1,exp_fit__9
Steps,,,
0,3104.865123,2744.287749,3626.647599
1,3040.725708,2698.426305,3537.818957
2,2977.487901,2652.960946,3450.606478
3,2915.139028,2607.888253,3364.980759
4,2853.666594,2563.204833,3280.912929
...,...,...,...
95,-269.020115,-236.524794,-402.405324
96,-285.732815,-256.642216,-417.928674
97,-302.210585,-276.585892,-433.169590


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 1,Exponential fit of node 9
Steps,,,
0,3104.865123,2744.287749,3626.647599
1,3040.725708,2698.426305,3537.818957
2,2977.487901,2652.960946,3450.606478
3,2915.139028,2607.888253,3364.980759
4,2853.666594,2563.204833,3280.912929
...,...,...,...
95,-269.020115,-236.524794,-402.405324
96,-285.732815,-256.642216,-417.928674
97,-302.210585,-276.585892,-433.169590


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}__{limits}.png",
    bbox_inches="tight",
)
plt.close()